In [1]:
from urllib.request import urlopen
import json
import pandas as pd
import geopandas as gpd
import plotly.express as px
import sys
import os


# set the parent directory to 2 directories up
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))
sys.path.append(parent_dir)

from src.GLOBAL import FIPS_MAPPING_DF

Root directory set to: /Users/jack/Documents/Projects/Power-Lab/Solar NIMBY/Solar-NIMBY-Analysis


In [6]:
datapath = parent_dir + "/data_processed/suitability_scores/"

# Load block group suitability scores
bg_suitability = pd.read_csv(datapath + "block_group_suitability_scores.csv", dtype={"GEOID": str})
bg_suitability = bg_suitability.rename(columns={"GEOID": "fips"})

# # Load geojson file for block groups
# with urlopen("https://opendata.dc.gov/api/download/v1/items/c143846b7bf4438c954c5bb28e5d1a21/geojson?layers=2") as response:
#     bg_geojson = json.load(response)
    
datapath = parent_dir + "/data/bounding_boxes/block_group_bounding_box_raw/"

bg_shapefile = gpd.read_file(datapath + "cb_2023_us_bg_500k.shp", dtype={"GEOID": str})
bg_json = bg_shapefile[['GEOID', 'geometry']].to_json()
bg_json = json.loads(bg_json)

# load a state level geojson file
with urlopen("https://raw.githubusercontent.com/PublicaMundi/MappingAPI/refs/heads/master/data/geojson/us-states.json") as response:
    states = json.load(response)

/Users/jack/anaconda3/envs/PowerLab/lib/python3.9/site-packages/pyogrio/raw.py:196: RuntimeWarning: driver ESRI Shapefile does not support open option DTYPE
  return ogr_read(


In [3]:
# Solar Data
datapath = parent_dir + "/data/electric_data/solar/"
solar = pd.read_csv(datapath + "solar_raw.csv")

# Get weighted average of all factors

**Formulas:**
1. Avg of all factors = (sum of all factors) / (number of factors)
2. Gem Model weighted avg
    - 2 slope
    - 1 land cover
    - 2 population density
    - 2 habitat
    - 4 protected land
    - 1 distance to substation
    - 4 GHI (in the default GEM they used `Solar PV: 1-Axis Tracking Flat-Plate Collector`)

In [4]:
# Formula 1
factors = ['GHI', 'Protected_Land', 'Habitat', 'Slope', 'Population_Density', 'Distance_to_Substation', 'Land_Cover']

# sum the factors
formula_1 = bg_suitability[factors].sum(axis=1).div(len(factors))
df_formula_1 = pd.DataFrame({"fips": bg_suitability['fips'], "suitability": formula_1})

# Formula 2
factors_plus_weights = {
    'GHI': 4,
    'Protected_Land': 4,
    'Habitat': 2,
    'Slope': 2,
    'Population_Density': 2,
    'Distance_to_Substation': 1,
    'Land_Cover': 1
}

formula_2 = bg_suitability.copy()
# Multiply each factor by its weight
for factor, weight in factors_plus_weights.items():
    formula_2[factor] = formula_2[factor] * weight
    
# sum the factors
formula_2 = formula_2[factors].sum(axis=1).div(sum(factors_plus_weights.values()))
df_formula_2 = pd.DataFrame({"fips": bg_suitability['fips'], "suitability": formula_2})

In [5]:
import json
from shapely.geometry import shape, mapping

def simplify_geojson(geojson_data, tolerance=0.01):
    """
    Simplifies the geometries in a GeoJSON file.

    :param input_path: Path to the input GeoJSON file.
    :param output_path: Path to save the simplified GeoJSON file.
    :param tolerance: The tolerance for simplification. Increase for more simplification.
    """

    for feature in geojson_data.get('features', []):
        geom = shape(feature['geometry'])
        # simplify the geometry; set preserve_topology=True to keep valid geometries
        simplified_geom = geom.simplify(tolerance, preserve_topology=True)
        feature['geometry'] = mapping(simplified_geom)

    return geojson_data

# Simplify the geojson
bg_json_simplified = simplify_geojson(bg_json, tolerance=0.05)

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

# custom color scale
pubu_color_scale = [
    (0.00, "#f7fcfd"),  # Very Light Blue (Unsuitable)
    (0.01, "#e0ecf4"), (0.40, "#e0ecf4"),  # Light Cyan
    (0.41, "#bfd3e6"), (0.50, "#bfd3e6"),  # Sky Blue
    (0.51, "#9ebcda"), (0.60, "#9ebcda"),  # Light Blue
    (0.61, "#8c96c6"), (0.70, "#8c96c6"),  # Moderate Blue
    (0.71, "#8c6bb1"), (0.80, "#8c6bb1"),  # Dark Teal
    (0.81, "#88419d"), (0.90, "#88419d"),  # Deep Purple-Blue
    (0.91, "#4a1486"), (1.00, "#4a1486")   # Very Dark Blue
]

fig = px.choropleth(
    df_formula_2,
    geojson=bg_json,
    locations='fips',  # Column in df that matches GeoJSON IDs
    featureidkey="properties.GEOID",  # Path to field in GeoJSON feature object with which to match the values passed in to locations
    color='suitability',  # Column to plot
    color_continuous_scale=pubu_color_scale,  # Custom color scale
    range_color=(0, 100),
    scope="usa",  # Focus on the United States    
)
fig.update_traces(marker_line_width=0, hoverinfo="skip")  # Remove bg borders

# Overlay state boundaries
fig.add_trace(go.Choropleth(
    geojson=states,  # Use the state-level GeoJSON
    locations=[feature["id"] for feature in states["features"]],  # State IDs
    z=[0] * len(states["features"]),  # Dummy data to avoid coloring
    colorscale=[[0, "rgba(0,0,0,0)"], [1, "rgba(0,0,0,0)"]],  # Transparent fill
    showscale=False,
    marker=dict(line=dict(color='black', width=1.5)),  # Black state boundaries
))

# Instead of adding individual scatter traces, create a more explicit legend for project sizes
fig.update_layout(
    coloraxis_colorbar=dict(
        title="Suitability Score",
        x=0.8,   # closer to the map’s right edge
        y=0.5,    # vertically centered
        len=1   # length of the colorbar (adjust as needed)
    ),
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
)

# Define representative sizes for the legend using max and min values from the solar data
size_values = [50, 100, 200, 300, 400, 500, 600]


# Add the actual solar projects as a single trace
scatter = px.scatter_geo(
    solar, 
    lat="latitude", 
    lon="longitude", 
    scope="usa", 
    size="total_mw",
    size_max=20,  # Control the maximum size of markers
    opacity=0.7,
    color_discrete_sequence=["rgba(255,127,0,0.8)"],  # Orange color for visibility
    hover_name="plant_name",
    hover_data=["total_mw", "statename"]
)

# Add this trace to the main figure
for trace in scatter.data:
    fig.add_trace(trace)

# Add reference points for the size legend (these points won't show on the map)
for size in size_values:
    fig.add_trace(go.Scattergeo(
        lat=[None],  # No actual points plotted
        lon=[None],
        mode='markers',
        marker=dict(
            size=np.sqrt(size) * 0.5,  # Scale size to match the actual points
            color='rgba(255,127,0,0.8)',  # Same color as actual points
            line=dict(width=1, color='black')
        ),
        name=f"{size} MW",
        legendgroup="sizes",
        showlegend=True
    ))

# Update layout to show both legends
fig.update_layout(
    showlegend=True,
    legend=dict(
        title="Solar Project Capacity",
        yanchor="top",
        xanchor="left",
        x=0.15,      # move horizontally (0 = left, 1 = right)
        y=0.95, 
        bgcolor="rgba(255,255,255,0.7)"
    )
)
# center the map
fig.update_geos(
    center=dict(lat=39.8283, lon=-98.5795),  # Approximate geographic center of contiguous US
    projection_scale=0.9,  # Adjust zoom level as needed (3.5-4.5 generally works well)
    visible=False
)

fig.update_layout(dragmode=False)
fig.show(renderer='notebook')
